***Can We Predict Gaming Duration?***

**Introduction**

This project is based on the data of collected by researchers at the Pacific Laboratory for Artificial Intelligence (PLAI) who set up a private server for Minecraft. Players who join the server have their actions recorded. The goal of PLAI is to create an embodied AI that can be interacted with like a real person in virtual worlds (e.g., Minecraft). Check out more [here](https://plaicraft.ai/).

We will be using the same dataset, but with a different goal. The goal of this project will be to study how different features in the dataset contribute to gaming time. 

More specifically, how do player experience level (veteran and above), age (under 20), and total hours played influence the duration of a game sessions?




To do so, the two datasets need to be merged and organized based on matching hashed emails. Then filter the data by experience level (veteran and up) and age (under 20). For non-numerical values, they will need to be encoded for the computer to read.

Since, one player may have multiple start and end times, we will first use played_hours variable from players.csv as the response variable first. 

**Methods & Results**

Linear regression will be the first model to test. This is because we believe that age and experience level may have a linear relationship with time played (e.g. if you're good at the game, you play more, or the younger you are the more time you have to play).  

However, linear regression will not be effective if the variables are not linearly correlated. If linear regression does not reveal a relationship, we will switch to classification using knn regression or knn. 

Also, we do have a lot of people who have not played the game at all. Using linear regression will skew the data, hence it may be worth to try filtering 0 played_hours out. This then poses the risk of the model not having enough data to learn. Using sessions to figure out played_time will be the next step. 

To fully explore the data, we will use both linear and knn regression as well as the K-nearest neighbor classification method. Results will be compared with metrics such as mpse and correlation coefficients. 

Data splitting may differ depending on the model we are testing, but for the linear regression model we are aiming for a 70% training and 30% testing splits. However, if we do decide to use cross-validation to test things out, we will start with a cv value of 5 or smaller as the dataset is small. 